In [19]:
import opendatasets as od
import pandas
 
od.download('https://www.kaggle.com/competitions/titanic/data')

Skipping, found downloaded files in ".\titanic" (use force=True to force download)


In [20]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

import numpy as np
import pandas as pd

In [21]:
train_data = pd.read_csv('Titanic/train.csv')
test_data = pd.read_csv('Titanic/test.csv')
gender_submission = pd.read_csv('Titanic/gender_submission.csv')

In [22]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [23]:
train_data.nunique()

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

In [24]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [25]:
train_data['Embarked'].fillna(value='S', inplace=True)
train_data = train_data.fillna(train_data['Fare'].mean())

test_data['Embarked'].fillna(value='S', inplace=True)
test_data = test_data.fillna(test_data['Fare'].mean())

data = [train_data, test_data]

for dataset in data:
    mean = train_data["Age"].mean()
    std = test_data["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = train_data["Age"].astype(int)
train_data["Age"].isnull().sum()

0

In [26]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,32.204208,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,32.204208,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,32.204208,S


In [27]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

train_data['Sex'] = le.fit_transform(train_data['Sex'])
train_data['Embarked'] = le.fit_transform(train_data['Embarked'])

test_data['Sex'] = le.fit_transform(test_data['Sex'])
test_data['Embarked'] = le.fit_transform(test_data['Embarked'])

In [28]:
y_train = train_data['Survived']
train_data = train_data.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)
test_data = test_data.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

# test_data.dropna(inplace=True)
# train_data.dropna(inplace=True)

y_train = train_data['Survived']
train_data = train_data.drop(['Survived'],axis=1)

In [29]:
# normalizing the data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data, y_train, test_size=0.3, random_state=42)

In [31]:

model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape = [7]),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(300, activation='relu'),
       tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Dense(300, activation='relu'),
       tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Dense(300, activation='relu'),
       tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Dense(100, activation='relu'),
       tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])

In [32]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])
model.fit(X_train,y_train,epochs=30)
# model.fit(train_data,y_train,epochs=50)

Epoch 1/30


c:\Program Files\Python310\lib\site-packages\keras\backend.py:5673: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


20/20 [==============================] - 1s 12ms/step - loss: 0.5818 - accuracy: 0.6870
Epoch 2/30
20/20 [==============================] - 0s 11ms/step - loss: 0.5110 - accuracy: 0.7753
Epoch 3/30
20/20 [==============================] - 0s 12ms/step - loss: 0.4666 - accuracy: 0.7929
Epoch 4/30
20/20 [==============================] - 0s 9ms/step - loss: 0.4563 - accuracy: 0.8010
Epoch 5/30
20/20 [==============================] - 0s 9ms/step - loss: 0.4496 - accuracy: 0.8122
Epoch 6/30
20/20 [==============================] - 0s 11ms/step - loss: 0.4403 - accuracy: 0.8042
Epoch 7/30
20/20 [==============================] - 0s 11ms/step - loss: 0.4204 - accuracy: 0.8315
Epoch 8/30
20/20 [==============================] - 0s 10ms/step - loss: 0.4079 - accuracy: 0.8283
Epoch 9/30
20/20 [==============================] - 0s 9ms/step - loss: 0.4319 - accuracy: 0.8186
Epoch 10/30
20/20 [==============================] - 0s 10ms/step - loss: 0.4133 - accuracy: 0.8218
Epoch 11/30
20/20 [====

In [33]:
predictions = model.evaluate(X_test, y_test, verbose=2)

9/9 - 0s - loss: 0.4761 - accuracy: 0.8134 - 173ms/epoch - 19ms/step


In [34]:
predictions = model.predict(test_data)

14/14 [==============================] - 0s 2ms/step


In [35]:
gender_submission['Survived'] = predictions.round().astype(int)

In [36]:
gender_submission.to_csv('Titanic/submission.csv',index=False)